In [13]:
#import Pkg;
#Pkg.add("Graphs")
#Pkg.add("GraphPlot")
#Pkg.add("Compose")
#Pkg.build("GraphPlot")
#Pkg.add("LightGraphs")
#Pkg.add("PyPlot")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
   Installed LaTeXStrings ─ v1.3.0
   Installed PyPlot ─────── v2.10.0
   Installed PyCall ─────── v1.93.1
    Updating `~/.julia/environments/v1.7/Project.toml`
  [d330b81b] + PyPlot v2.10.0
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [b964fa9f] + LaTeXStrings v1.3.0
  [438e738f] + PyCall v1.93.1
  [d330b81b] + PyPlot v2.10.0
    Building PyCall → `~

In [5]:
using DelimitedFiles
using Graphs, GraphPlot #, Compose, LightGraphs, PyPlot

In [4]:
struct Instance
    graph::SimpleDiGraph             #graphe du probleme 
    vertex_weight::Vector{Float64}   #poids associés au noeuds
    edge_weight::Matrix{Float64}     #poids associés au arcs
    pairs::Vector{Int}               #ensemble des noeuds correspondant aux paires de personnes
    altruists::Vector{Int}           #ensemble des noeuds altruistes
    nb_pairs::Int                    #nombre de noeuds correspondant aux paires de personnes
    nb_altruists::Int                #nombre de noeuds altruistes
    max_cycle_length::Int            #longueur maximale de cycle toléré
    max_chain_length::Int            #longueur de la chaine de la solution (si j'ai bien compris) 
    is_vertex_weighted::Bool         #booléen specifiant si les arcs sont pondérés ou non
    
    #constructeur 1 (graph + poids en entrée)
    function Instance(g::SimpleDiGraph, vertex_weight::Array{Float64}, K::Int, L::Int = 0)
        P = [v for v in vertices(g) if vertex_weight[v] != 0.0] #ensemble des noeuds ponderés (paires)
        A = [v for v in vertices(g) if vertex_weight[v] == 0.0] #ensemble des noeuds non ponderés (altruistes)
        edge_weight = zeros(nv(g), nv(g)) #initialisation du poids des arcs a 0
        for e in edges(g)
            edge_weight[e.src,e.dst] = vertex_weight[e.dst] #pour tous les noeuds, on definit le poids d'un arc par le poids du noeud d'arrive
        end

        return new(g, vertex_weight, edge_weight, P, A, length(P), length(A), K, L, true)
        #retourne les attributs de l'instance
                                    
    end
           
    #contructeur 2 (fichier en entrée)
    function Instance(filename::String, K::Int, L::Int = 0)
        inst = string(filename)
        data_folder = "data" # joinpath(pkgdir(KidneyExchange), "data")
        wmd_file = joinpath(data_folder, join([inst, ".wmd"]))
        dat_file = joinpath(data_folder, join([inst, ".dat"]))

        g, edge_weight, is_altruist = read_kep_file(wmd_file, dat_file) #appel a la fonction du dessous
        P = [v for v in vertices(g) if !is_altruist[v]] #ensemble des noeuds correspondant aux paires de personnes
        A = [v for v in vertices(g) if is_altruist[v]]  #ensemble des noeuds altruites
        vertex_weight = zeros(nv(g)) #initialisation des poids des noeuds
        is_vertex_weighted = true
        #creation des poids des noeuds
        for v in P
            vertex_weight[v] = 1.0
            if indegree(g, v) >= 1
                vertex_weight[v] = edge_weight[inneighbors(g,v)[1],v]
                for u in inneighbors(g,v)
                    if edge_weight[u,v] != vertex_weight[v]
                        is_vertex_weighted = false
                        break
                    end
                end
                if !is_vertex_weighted  break   end
            end
        end
        if !is_vertex_weighted
            println("the instance is not vertex weighted!")
        end
        return new(g, vertex_weight, edge_weight, P, A, length(P), length(A), K, L, is_vertex_weighted)
            #retourne les attributs de l'instance
    end 
                            
end

LoadError: UndefVarError: SimpleDiGraph not defined

In [2]:
#fonction qui prend en parametre les deux fichiers et retourne le graphe associé, le poids des arcs et le vecteur indicateur des altruistes
function read_kep_file(wmd_file::AbstractString, dat_file::AbstractString)

    #wmd_file_name = split(split(wmd_file, '/')[end], '.')[1]
    #dat_file_name = split(split(dat_file, '/')[end], '.')[1]

	wmd_file_name = first(splitext(last(splitpath(wmd_file))))
	dat_file_name = first(splitext(last(splitpath(dat_file))))

    wmd_file_name == dat_file_name || throw(ArgumentError(".wmd and .dat files do not correspond to the same dataset."))
	isfile(abspath(wmd_file)) || throw(ArgumentError("$(abspath(wmd_file)): file not found."))
	isfile(abspath(dat_file)) || throw(ArgumentError(".dat file not found."))

	# Get the number of vertices and edges from the first line of wmd file
    wmd_io = open(wmd_file, "r")
	splitted_line = split(readline(wmd_io), ',')
	nb_vertices = parse(Int, splitted_line[1])
	nb_edges = parse(Int, splitted_line[2])

    # Extract meta information from the .dat file (atruiste ou non )
    file = readdlm(dat_file, '\n')
	is_altruist = falses(nb_vertices)
	ind = 1
    for line in file[2:end]
        splitted_line = split(line, ',')
        if Bool(parse(Int, splitted_line[7]))
			is_altruist[ind] = true
		end
		ind += 1
    end

    # Extract the graph structure from the .wmd file
    # skip next nb_vertices lines, which are redundant with the data contained in the .dat file
    for i in 1:nb_vertices
        readline(wmd_io)
    end

    # read the set of edges
	in_list = Vector{Vector{Int}}(undef, nb_vertices)
	out_list = Vector{Vector{Int}}(undef, nb_vertices)
	for u in 1:nb_vertices
		in_list[u] = Vector{Int}()
		out_list[u] = Vector{Int}()
	end
	ne = 0
	edge_weight = zeros(nb_vertices, nb_vertices)
    while !eof(wmd_io)
        splitted_line = split(readline(wmd_io), ',')
        # /!\ Pairs are numbered from 0 in the second part of the file
        src = parse(Int, splitted_line[1]) + 1
        dst = parse(Int, splitted_line[2]) + 1
        weight = parse(Float64, splitted_line[3])

        # do not add an edge that has an altruist as destination or that has a zero weight
		if !is_altruist[dst] && weight > 0.0
			ne += 1
			push!(out_list[src], dst)
			push!(in_list[dst], src)
			edge_weight[src, dst] = weight
		end
    end
	for u in 1:nb_vertices
		sort!(out_list[u])
		sort!(in_list[u])
	end
    return SimpleDiGraph(ne, out_list, in_list), edge_weight, is_altruist
end

read_kep_file (generic function with 1 method)

In [6]:
#Instance de test
data_folder = string(@__DIR__,"/data")
instance_name = "MD-00001-00000010"
filename = joinpath(data_folder, join([instance_name]))
new_inst = Instance(filename, 3, 0)

LoadError: MethodError: no method matching Instance(::String, ::Int64, ::Int64)

In [8]:
#Affichage du graphe (difficile de faire un affichage propre)
g = new_inst.graph

x = [0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10, 0, 0, 0, 0]
y = [0, 0, 0, 0, 0, 10, 20, 30, 40, 40, 40, 40, 40, 30, 20, 10]

nodelabel = new_inst.pairs
gplot(g, nodelabel=nodelabel, x, y)


LoadError: UndefVarError: new_inst not defined

In [7]:
#new_inst.graph : graphique

"The instance is represented by the simple directed graph: $(new_inst.graph)"

"The instance is represented by the simple directed graph: {16, 47} directed simple Int64 graph"

In [10]:
#new_inst.vertex_weight : vecteur des poids des noeuds

"Vertex weights are given as $(new_inst.vertex_weight)"

"Vertex weights are given as [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"

In [11]:
#new_inst.edge_weight : matrice des poids des arcs

new_inst.edge_weight[1,2]

0.0

In [52]:
#new_inst.pairs : ensemble des noeuds representant une paire

new_inst.pairs

16

In [42]:
#new_inst.altruists : ensemble des noeuds representant un altruiste

new_inst.altruists

0-element Array{Int64,1}

In [43]:
#new_inst.is_vertex_weighted : booleen indiquant si les noeuds sont pondérés

new_inst.is_vertex_weighted

true

In [60]:
#nv(new_inst.graph) : nombre de noeuds du graphe
#new_inst.nb_pairs : nombre de noeuds correspondant a une paire
#new_inst.nb_altruists : nombre de noeuds correspondant a un altruiste

print("Nombre de paires : ", new_inst.nb_pairs,"\n")
print("Nombre d'altruistes : ",new_inst.nb_altruists,"\n")
"This graph has $(nv(new_inst.graph)) nodes."


Nombre de paires : 16
Nombre d'altruistes : 0


"This graph has 16 nodes."

In [25]:
#ne(new_inst.graph) : nombre d'arcs du graphe

"This graph has $(ne(new_inst.graph)) arcs."

"This graph has 47 arcs"

In [31]:


"The set of nodes is given by $(vertices(new_inst.graph)). This is an iterable object containing all vertices."

"The set of nodes is given by Base.OneTo(16). This is an iterable object containing all arcs."

In [61]:
for v in vertices(new_inst.graph)
    println(v)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [28]:
"The set of arcs is given by $(edges(new_inst.graph)). This is an iterable object containing all arcs."

"The set of arcs is given by SimpleEdgeIter 47"

In [1]:
#soit e un arc 
#src(e) : sommet origine de l'arc
#dst(e) : sommet destination de l'arc

for e in edges(new_inst.graph)
    println("$e -- has $(src(e)) as its origin and $(dst(e)) as its destination")
end

LoadError: UndefVarError: new_inst not defined